In [180]:
using LinearAlgebra
using Printf
Base.show(io::IO, f::Float64) = @printf(io, "%1.3f", f)

## _Primitivas_

### Questão (1)
Implementação de uma função que calcula a norma de um vetor no $R^n$.

In [168]:
#Norma Comum
function NormaComumdoVetor(V)
    sumV=0
    for i in V #Somatório do Elemento
        sumV = sumV + i*i
    end
    NormaV = sqrt(sumV)

    return NormaV
end

#Norma W - Recebe um vetor V e retorna a versão vetor W definida na questão
function NormaWdoVetor(V)
    Beta =  maximum(V) #Beta é o maior valor do vetor V
    W = (1/Beta)*V # o vetor W é o produto entre inversa de Beta e vetor V
    sumW=0
    for i in W #Somatório do Elemento
        sumW = sumW + i*i
    end
    NormaW = sqrt(sumW)

    return NormaW
end

NormaWdoVetor (generic function with 1 method)

### Questão (2) Rotações de Givens - Caso Real

Implementação de uma função que recebe dois números reais, f e g, e calcula *c,s* e *r* com as propriedades de norma implementadas acima, sem usar funções trigonométricas na resposta final.

In [169]:
#Função Rotação de Givens, recebe a e b, e retorna r, c, s
function RotacaoDeGivens2x2(f,g)

    r = sqrt((f*f)+(g*g)) #Achando o valor de r
    c = f/r #Fórmula do valor de c
    s = g/r #Fórmula do valor de s

    return r,c,s
end

RotacaoDeGivens2x2 (generic function with 1 method)

### Questão (3) Refletores de Householder - Caso Real

Função que recebe um vetor w e computa γ e v com as propriedades implementadas acima.

In [170]:
#Função Reflexão de Householder, recebe vetor W e retorna 
function ReflexaoDeHouseholder(W)
    
    SizeW = length(W) #Pegando o tamanho do Vetor W
    NormadeW = NormaComumdoVetor(W) #Já tirando a norma de W
    
    e = zeros(SizeW, 1) #Aqui criaremos o vetor elementar e do mesmo tamanho do W
    e[1] = 1 #O primeiro valor de e é 1
    
    α = NormadeW #α é a norma de W

    V = W - α*e
    print("V:\n", V)
    
    NormadeV = NormaComumdoVetor(V) #Norma do V encontrado para acharmos o γ
    γ = 2/(NormadeV*NormadeV)
    print(V)
    return γ, V
end

ReflexaoDeHouseholder (generic function with 1 method)

In [171]:
#TESTE
γ, V = ReflexaoDeHouseholder([3; 4])
P = I - γ*V*(V')
P*[3; 4]

V:
[-2.000; 4.000][-2.000; 4.000]

2-element Array{Float64,1}:
 5.000
 0.000

### Questão (4) Decomposição QR

Implementação de uma função que calcula a decomposição QR de uma matriz A:

In [181]:
#QR por Givens
function QR2x2porRotacaoDeGivens(A) 
    V = A[:,1]
    a = A[1]
    b = A[2]
    r,c,s = RotacaoDeGivens2x2Givens(a,b)
    Q = [c -s ; s c]
    return Q
end

#QR por Reflexão de Householder
function QRporReflexaoDeHouseholder(A) 
    Q=one(A)
    mA = size(A,1)
    nA = size(A,2)
    for i in 1:(nA-1)
        vtemp = A[i:mA,i] #vetor temporario que é enviado pra Reflexao
        print(size(vtemp))
        γ, v = ReflexaoDeHouseholder(vtemp)
        vt = transpose(v) 
        P = I - γ*v*vt #formula de Householder
        Qi = P
        Q[:,i:nA] = Q[:,i:nA]*Qi
        A[i:mA, i:nA]=P*A[i:mA, i:nA] #Problema aqui em alocar os valores das submatrizes, nao tenho ideia de como fazer em julia por hora
    end
    return Q,A
end

QRporReflexaoDeHouseholder (generic function with 1 method)

In [184]:
#TESTE
#QRpronto(A)
function QRpronto(A) 
    Q,R = qr(A)
    print(R)
end

QRpronto([6.0 5.0 0.0;
        5.0 1.0 4.0;
        0.0 4.0 3.0])

[-7.810 -4.481 -2.561; 0.000 4.682 0.966; 0.000 0.000 4.184]

In [188]:
#TESTE
Q, R = QRporReflexaoDeHouseholder(
        [6.0 5.0 0.0;
        5.0 1.0 4.0;
        0.0 4.0 3.0])

println("\nMatriz Triangular R:", R, "\n")
println(Q*R, "\n")
println(Q'*Q, "\n")

(3,)V:
[-1.810; 5.000; 0.000][-1.810; 5.000; 0.000](2,)V:
[-2.249; 4.000][-2.249; 4.000]
Matriz Triangular R:[7.810 4.481 2.561; -0.000 4.682 0.966; 0.000 0.000 -4.184]

[6.000 5.000 0.000; 5.000 1.000 4.000; -0.000 4.000 3.000]

[1.000 0.000 0.000; 0.000 1.000 0.000; 0.000 0.000 1.000]



## _Autovalores e autovetores_

### Questão (5) Redução à forma de Hessenberg

Implementação de uma função que reduza uma matriz A a forma de Hessenberg:

In [193]:
#Redução a forma de Hessenberg
function ReducaoAHessenberg(A) 
    Q = one(A)
    mA = size(A,1)
    nA = size(A,2)
    for i in 1:(nA-2)
        vtemp = A[i:mA,i]
        γ, v = ReflexaoDeHouseholder(vtemp)
        P = I - γ*v*v' 
        Qi = P
        Q[:,i:nA] = Q[:,i:nA]*Qi
        A[i:mA, i:nA]=P*A[i:mA, i:nA]
        print("\n")
        print("MATRIZ Q:\n",Q)
    end
    return H,Q
end

ReducaoAHessenberg (generic function with 1 method)

In [194]:
#TESTE
H, Q = ReducaoAHessenberg(
        [6.0 5.0 0.0;
        5.0 1.0 4.0;
        0.0 4.0 3.0])

V:
[-1.810; 5.000; 0.000][-1.810; 5.000; 0.000]
MATRIZ Q:
[0.768 0.640 0.000; 0.640 -0.768 0.000; 0.000 0.000 1.000]

([0.768 0.333 0.547; 0.640 -0.399 -0.656; 0.000 0.854 -0.520], [0.768 0.640 0.000; 0.640 -0.768 0.000; 0.000 0.000 1.000])

### Questão (6) Iteração de Francis de grau 2 

In [195]:
using LinearAlgebra

#Iteração de Francis
function IteraçãodeFrancis(H,Q) 
    Qx = conjugate(Q') #Q conjugado e transposto
    A = Q*H*Qx
    R = H*Qx
    p(H) = (H-p1*I)*(H-p2*I) #p1 e p2 são shifts a escolher
    Hchapeu = H = α*p(H)
    return Hchapeu
end

IteraçãodeFrancis (generic function with 1 method)

### Questão (7) Shift de Rayleigh Generalizado

Implemente uma função que calcula os shifts que usaremos: 
O shift de Rayleigh generalizado (dois autovalores da matriz $H_{canto}$) se eles não forem reais, ou duas cópias do shift de Wilkinson (autovalores da matriz $H_{canto}$ mais proximo de $h_{n,n}$) caso contrário

In [196]:
using LinearAlgebra

#Shift de Raileigh para achar os autovalores p1 e p2 da matriz Hcanto
function ShiftDeRaileigh(H) 
    
    return p1, p2
end

ShiftDeRaileigh (generic function with 1 method)

### Questão (8) Aplicando $Q_0$ tal que $Q_0^1(p(H)e_1) = βe_1$

Encontre manualmente uma formula explícita para o vetor $(H − ρ_1I)(H − ρ_2I)e_1$ usando que H e Hessenberg. Use sua função da seção 1.3 para encontrar um refletor $Q_0$ pequeno (que opera em poucas linhas/colunas) com a propriedade desejada.

In [197]:
using LinearAlgebra

function EncontreQ0(H) 
    
    return Q0
end

EncontreQ0 (generic function with 1 method)

### Questão Extra - Voltando a forma de Hessenberg
Se A é simétrica, H é tridiagonal, pois uma matriz similar a uma simétrica é também simétrica. Se não quisermos calcular Q, implemente uma otimização que faça uma iteração de Francis (no caso simétrico) em tempo O(n) e verifique que sua função se comporta igual ao caso geral.
Dica: Aplicar os refletores do algoritmo numa matriz tridiagonal demora tempo O(1), pois, para um dado refletor, quase todas as linhas/colunas conterão zeros nas posicões relevantes.

In [109]:
using LinearAlgebra

function FrancisOdeN(A,H) 
    numColH = size(H,2)
    for i in 1:numColH
        Wtemp = H[:,i]
        γ, v = ReflexaoDeHouseholder(Wtemp)
        Qi = I - γ*v*vt #Falta consertar a parte de guardar os valores do Qi
    end
    return Q0
end

FrancisOdeN (generic function with 1 method)

### Questão (9) Juntando tudo no caso simétrico
Implemente o procedimento acima para fazer varias iterações de Francis. Faça seu código imprimir uma mensagem dizendo qual dos dois casos (Caso 1 ou Caso 2) reduziu o problema. Qual caso acontece mais frequentemente?

In [110]:
println("Hello World")

Hello World


### Questão Extra - Juntando tudo no caso simétrico
Para todo $k = 1, . . . , n$, verifique numericamente que $p(H)e_k$ pode ser expresso como combinação linear das colunas ${q1, . . . , qk}$, e portanto que estamos iterando subespaços. Lembre-se de se restringir ao pedaço da matriz que e Hessenberg próprio.

In [18]:
println("Hello World")

Hello World


## _SVD de matrizes reais $n \times m$ com $n \geqslant m$_

### Questão (10) Bidiagonalização de Golub–Kahan

Implemente o algoritmo de bidiagonalização de Golub–Kahan.

In [19]:
println("Hello World")

Hello World


### Questão (11) Golub–Reinsch: SVD de B bidiagonal própria

Implemente uma iteração do algoritmo de Golub–Reinsch

In [20]:
println("Hello World")

Hello World
